# Analyse badlands stratigraphic output

If the stratigraphic structure is turned on in the XmL input file, **badlands** produces sedimentary layers Hdf5 files. 

> The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [ ]:
import numpy as np
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import scipy.ndimage.filters as filters
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter

import stratalAnalyse as strata

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

# Loading stratigraphic file

First we need to load the stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.hdf5`

with T the display time index.

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
# help(strata.stratalSection.__init__)

In [ ]:
folder = 'output/h5'
strat = strata.stratalSection(folder)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
# help(strat.loadStratigraphy)

In [ ]:
strat.loadStratigraphy(100) 

# Building a cross-section

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide:
+ the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, 
+ a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and 
+ the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Build a cross-section along X axis
x1 = 0.
x2 = 300000.
y1 = 100000
y2 = 100000

# Interpolation parameters
nbpts = 301
gfilt = 1

In [ ]:
# help(strat.buildSection)

In [ ]:
strat.buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

# Stratal stacking pattern

We use **plotly** to visualise the vertival cross-section of stratal stacking apttern.

In [ ]:
#help(strata.viewSection)

In [ ]:
strata.viewSection(width = 1000, height = 500, cs = strat, 
            dnlay = 2, rangeX=[220000, 300000], rangeY=[-1050,-50],
            linesize = 0.5, title='Stratal stacking pattern coloured by time')

# Sequence stratigraphy methods proposed in this notebook

New analytical methods are proposed in recent years on the interpretation of depositional systems. Here, we apply three methods, including 
+ **(i) the systems tracts method** in which the declaration of stratigraphic sequences is based on relative sea-level change; 
+ **(ii) the shoreline trajectory analysis** that defines different trajectory classes according to the migration of shoreline; 
+ **(iii) the accommodation sucession method** that focuses on the objective observation of depositional trends and then defines different sequence sets reponding to the competing ration between the change of accommodation and sediment supply.

# 1- Systems Tracts Method - based on relative sea-level

There are several models of systems tracts within depositional sequences, here we use the most simple one called the four systems tract model (figure is from [Helland-Hansen & Hampson (2009)](http://onlinelibrary.wiley.com/wol1/doi/10.1111/j.1365-2117.2009.00425.x/full)): <img src="images/SystemsTract_RSL.png" alt="Systems tract defined based on relative sea-level"  width="300" height="300" border="10" />
- highstand systems tract **HST** 
- falling-stage systems tract **FSST**
- lowstand systems tract **LST**
- transgressive systems tract **TST**

For each of these system tracts we define a given color. We use the 'colorlover' library [link](http://moderndata.plot.ly/color-scales-in-ipython-notebook/)

## Relative base-level

We first visualize the sea-level curve, and then extract **manually** the time intervals that bound different systems tracts, as showed in the above figure.

In [ ]:
time, Sealevel = strata.readSea('data/sealevel.csv')
timeStep = 1e5

# Plot sea-level
strata.viewData(x0 = time/timeStep, y0 = Sealevel, width = 1000, height = 500, linesize = 3, 
                color = '#6666FF',title='Sea-level curve',xlegend='display steps',ylegend='sea-level position')

We will assign 4 different colors based on relative sea-level change. To visualise the colors you can copy the html code below in the following [website](https://www.w3schools.com/cssref/tryit.asp?filename=trycss_color_rgba):

```html
<!DOCTYPE html><html>
  <head>
	<style>
      #p1 {background-color:rgba(213,139,214,0.8);}
      #p2 {background-color:rgba(215,171,117,0.8);}
      #p3 {background-color:rgba(39,123,70,0.8);}
      #p4 {background-color:rgba(82,128,233,0.8);}
    </style>
  </head>
  <body>
    <p>RGB colors with opacity for Systems-Tracts:</p>
    <p id="p1">HST</p>
    <p id="p2">FSST</p>
    <p id="p3">LST</p>
    <p id="p4">TST</p>
  </body>
</html>
```
Define different colors for different systems tracts

In [ ]:
cHST = 'rgba(213,139,214,0.8)' 
cFSST = 'rgba(215,171,117,0.8)' 
cLST = 'rgba(39,123,70,0.8)' 
cTST = 'rgba(82,128,233,0.8)' 

Specify the extent of every systems tracts according to the relative sea-level change

In [ ]:
HST1 = np.array([0,11],dtype=int)
FSST1 = np.array([11,26],dtype=int)
LST1 = np.array([26,30],dtype=int)
TST1 = np.array([30,36],dtype=int)
HST2 = np.array([36,42],dtype=int)
FSST2 = np.array([42,69],dtype=int)
LST2 = np.array([69,78],dtype=int)
TST2 = np.array([78,88],dtype=int)
HST3 = np.array([88,100],dtype=int)

Build the color list for each systems tract

In [ ]:
# Build the color list
STcolors = []
for k in range(HST1[0],HST1[1]):
    STcolors.append(cHST)
for k in range(FSST1[0],FSST1[1]):
    STcolors.append(cFSST)
for k in range(LST1[0],LST1[1]):
    STcolors.append(cLST)
for k in range(TST1[0],TST1[1]):
    STcolors.append(cTST)
for k in range(HST2[0],HST2[1]):
    STcolors.append(cHST)
for k in range(FSST2[0],FSST2[1]):
    STcolors.append(cFSST)
for k in range(LST2[0],LST2[1]):
    STcolors.append(cLST)
for k in range(TST2[0],TST2[1]):
    STcolors.append(cTST)
for k in range(HST3[0],HST3[1]):
    STcolors.append(cHST)
STcolors.append(cHST)

## Plotting the resulting systems tracts

In [ ]:
# help(strata.viewSectionST)

In [ ]:
strata.viewSectionST(width = 1000, height = 500, cs=strat, colors=STcolors,
                   dnlay = 2, rangeX=[220000, 300000], rangeY=[-1050,-50], 
                   linesize=0.5, title='Systems tracts interpreted based on relative sea-level (RSL)')